このノートブックではOptunaの使い方を確認します.

In [1]:
from typing import Any, Dict
import category_encoders as ce
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import cross_val_score
from catboost import CatBoostClassifier
from pandas import DataFrame
import optuna

df = sns.load_dataset('titanic')
df.head()

# 必要な特徴量を抽出
feature_names = [
    'class',
    'sex',
    'age',
    'sibsp',
    'parch',
    'fare',
    'embark_town',
    'deck',
]
df_x = df[feature_names]
df_y = df['survived']


データを訓練データとテストデータに分割します. テストデータはハイパーパラメータ最適化には使用せず, 最適なハイパーパラメータで訓練されたモデルを評価するために使用されます.

In [2]:
x_tr, x_te, y_tr, y_te = train_test_split(df_x, df_y, test_size=0.33, shuffle=True, random_state=42)

Optunaは目的関数に対してベイズ最適化を適用します.
目的関数はハイパーパラメータを受け取り, そのハイパーパラメータで訓練されたモデルの性能指標を返します.
まず, Catboostのハイパーパラメータの探索範囲を決める関数を作成します.

In [3]:
def suggest_params(trial: optuna.Trial) -> Dict:
    # Catboostの繰り返し回数
    iterations = trial.suggest_categorical("iterations", [200, 1000])

    # cat_featuresは本来は列名のリストとして与えられますが,
    # ここではログを表示した時の見やすさのために仮の値をセットしています.
    cat_features = trial.suggest_categorical("cat_features", ["none", "given"])

    return {
        "iterations": iterations,
        "cat_features": cat_features,
    }

次に, 与えられたデータに対して目的関数を返す関数を定義します.

In [4]:
def create_objective(x: DataFrame, y: DataFrame) -> Any:  # 戻り値の型は後で書く
    cols = ['class', 'sex', 'embark_town', 'deck']

    # 各列に全ての取り得るユニークな値が含まれるとは限らないため良くない方法ですが
    # 簡単さのためにこのようにしています
    encoder = ce.OrdinalEncoder(cols=cols, handle_unknown='impute')

    def objective(trial: optuna.Trial) -> Any:  # 戻り値の型は後で書く
        params = suggest_params(trial)

        # cat_featuresを正しい値に設定
        cat_features = None if params["cat_features"] == "none" else cols
        params["cat_features"] = cat_features

        # パイプラインを構成
        clf = CatBoostClassifier(
            **params,
            verbose=False
        )
        pipe = make_pipeline(encoder, clf)

        # スコアを計算
        score = cross_val_score(pipe, x, y, cv=5).mean()

        return score

    return objective


ベイズ最適化を実行します.

In [7]:
objective = create_objective(df_x, df_y)
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10, show_progress_bar=True)


[I 2022-06-26 07:49:41,704] A new study created in memory with name: no-name-15016f02-c22d-4ccc-bc1c-58fc84b71e15
/Users/taku-y/git/github/PyDataOsaka/learn-mlflow/venv/lib/python3.9/site-packages/optuna/progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2022-06-26 07:49:47,321] Trial 0 finished with value: 0.8159374803841567 and parameters: {'iterations': 1000, 'cat_features': 'given'}. Best is trial 0 with value: 0.8159374803841567.
[I 2022-06-26 07:49:47,980] Trial 1 finished with value: 0.8226916075575922 and parameters: {'iterations': 200, 'cat_features': 'none'}. Best is trial 1 with value: 0.8226916075575922.
[I 2022-06-26 07:49:53,331] Trial 2 finished with value: 0.8159374803841567 and parameters: {'iterations': 1000, 'cat_features': 'given'}. Best is trial 1 with value: 0.8226916075575922.
[I 2022-06-26 07:49:54,042] Trial 3 finished with value: 0.8226916075575922 and parameters: {'iterations': 200, 'cat_features': 'none'}. Best is trial 1 with value: 0.8226916075575922.
[I 2022-06-26 07:49:54,723] Trial 4 finished with value: 0.8226916075575922 and parameters: {'iterations': 200, 'cat_features': 'none'}. Best is trial 1 with value: 0.8226916075575922.
[I 2022-06-26 07:49:55,370] Trial 5 finished with value: 0.822691607557